In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pwd

/content


In [ ]:
%cd /content/drive/Shareddrives/CSE573/preprocessed
%ls

/content/drive/Shareddrives/CSE573/preprocessed
AaplAmzn_NewsDateDirection.csv
Aapl_NewsPriceHistDirection.csv
Aapl_SentimentAggMeanPriceHistDirection.csv
Aapl_SentimentAggSumPriceHistDirection.csv
Amzn_NewsPriceHistDirection.csv
Amzn_SentimentAggMeanPriceHistDirection.csv
Amzn_SentimentAggSumPriceHistDirection.csv
amzn_title_processedText_direction.csv
amzn_title_text_label.csv


In [ ]:
#!pip install keras
#!pip install matplotlib-venn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from collections import Counter
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

import requests
import io

In [ ]:
df_amzn = pd.read_csv('Amzn_NewsPriceHistDirection.csv')
df_amzn

,uuid,author,url,title,language,text,published,title_aapl,text_aapl,title_amzn,...,prevClose_16,prevClose_17,prevClose_18,prevClose_19,prevClose_20,prevClose_21,prevClose_22,prevClose_23,prevClose_24,prevClose_25
0,3b7327f6858be72e780b02b863d941fe7ce81eab,NaN,https://money.cnn.com/2018/09/05/investing/pre...,6 things to know before the opening bell,english,Click chart for more in-depth data. 1. Big Tec...,2018-09-05 05:12:00+00:00,False,True,False,...,1896.99,1885.99,1896.00,1882.98,1861.48,1847.11,1822.94,1833.98,1796.14,1787.38
1,8a3015a274df99d231fcf7ff3eb38e2660a984ff,Michael Wiggins De Oliveira,https://seekingalpha.com/article/4203857-netfl...,"Netflix And The Competition - Netflix, Inc. (N...",english,Investment Thesis Netflix ( NFLX ) is a remark...,2018-09-04 15:00:00+00:00,False,True,False,...,1896.99,1885.99,1896.00,1882.98,1861.48,1847.11,1822.94,1833.98,1796.14,1787.38
2,df5883d078ebfbe015e1e717002e4b889647dbee,Larry Ramer,https://investorplace.com/2018/09/pandora-stoc...,Pandora Stock Gets a Boost — Buy Now,english,MannKind Surges More Than 50% >>> READ MORE\nB...,2018-09-04 16:22:00+00:00,False,True,False,...,1896.99,1885.99,1896.00,1882.98,1861.48,1847.11,1822.94,1833.98,1796.14,1787.38
3,861c19bff56d09f97b6798f5fbe374061e8aa55b,morningstar.com,http://news.morningstar.com/all/dow-jones/us-m...,MARKET SNAPSHOT: Tech Tumble Weighs On Stocks;...,english,"By Mark DeCambre, MarketWatch , Ryan Vlastelic...",2018-09-05 00:00:00+00:00,False,True,False,...,1896.99,1885.99,1896.00,1882.98,1861.48,1847.11,1822.94,1833.98,1796.14,1787.38
4,cb960bebc2bf78d327026127c9fe919a4159e80d,morningstar.com,http://news.morningstar.com/all/benzinga/BenzB...,"New Month, Same Old Story: Trade Fears Persist...",english,"It may be a new month, but the same trade conc...",2018-09-05 00:00:00+00:00,False,True,False,...,1896.99,1885.99,1896.00,1882.98,1861.48,1847.11,1822.94,1833.98,1796.14,1787.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19074,9f9c98efda2cab35c89c5284a03a6e12b3ba35aa,zacks.com,https://www.zacks.com/stock/news/308958/micros...,"Microsoft Roundup: Hardware Leak, Cashier-less...",english,Leaked documents showed what could be Microsof...,2018-06-22 00:00:00+00:00,False,True,False,...,1630.70,1623.58,1613.02,1609.39,1602.73,1600.50,1580.75,1584.00,1574.20,1580.63
19075,38fa20acb3ab406d477b460698f4974a91ebbb13,Daniel Jones,https://seekingalpha.com/article/4183134-gener...,General Electric's Fall From Grace: Don't Fret...,english,June 19th was a dark day in the history books ...,2018-06-21 18:15:00+00:00,False,True,False,...,1630.70,1623.58,1613.02,1609.39,1602.73,1600.50,1580.75,1584.00,1574.20,1580.63
19076,2ec038a09f5084fcec9d930d325e6ff761698a25,morningstar.com,http://news.morningstar.com/all/market-watch/T...,"Beware of the Russell 2000, but should you buy...",english,"By Thomas H. Kee Jr.\nThe Russell 2000 is hot,...",2018-06-22 00:00:00+00:00,False,True,False,...,1630.70,1623.58,1613.02,1609.39,1602.73,1600.50,1580.75,1584.00,1574.20,1580.63
19077,74ad3bc86b356f34f8a7f2f477f5827c789fed33,Thomas H. Kee Jr.,https://www.marketwatch.com/story/beware-of-th...,"Beware of the Russell 2000, but should you buy...",english,There has been a bifurcation in the U.S. stock...,2018-06-22 00:00:00+00:00,False,True,False,...,1630.70,1623.58,1613.02,1609.39,1602.73,1600.50,1580.75,1584.00,1574.20,1580.63


In [ ]:
df_amzn = df_amzn[["title", "text", "direction"]]

In [ ]:
df_amzn.head()

,title,text,direction
0,6 things to know before the opening bell,Click chart for more in-depth data. 1. Big Tec...,-1
1,"Netflix And The Competition - Netflix, Inc. (N...",Investment Thesis Netflix ( NFLX ) is a remark...,-1
2,Pandora Stock Gets a Boost — Buy Now,MannKind Surges More Than 50% >>> READ MORE\nB...,-1
3,MARKET SNAPSHOT: Tech Tumble Weighs On Stocks;...,"By Mark DeCambre, MarketWatch , Ryan Vlastelic...",-1
4,"New Month, Same Old Story: Trade Fears Persist...","It may be a new month, but the same trade conc...",-1


In [ ]:
df_amzn['text'] = df_amzn['text'].apply(lambda x: x.lower())

In [ ]:
df_amzn['text'][0]

"click chart for more in-depth data. 1. big tech in senate: big tech executives are coming to capitol hill on wednesday to testify about election meddling. facebook ( fb ) coo sheryl sandberg and jack dorsey, the ceo of twitter ( twtr ) , will appear before the senate intelligence committee to discuss actions their companies have taken to thwart foreign influence campaigns targeting the 2018 midterm elections.\nthe committee requested that a high-profile executive from google, which owns youtube, also appear to testify. google ( googl ) and its parent company alphabet, however, declined to send one of their top people.\n2. emerging markets fears: emerging markets currencies tumbled against the dollar on wednesday, with the mexican peso, south african rand, indian rupee and indonesian rupiah among the biggest losers.\nrising us interest rates, political clashes and the global trade war have in recent months exposed frailties in multiple emerging markets. turkey and argentina have been h

In [ ]:
import string

In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
#stopwords.words('english')
df_amzn['text_without_stopwords'] = df_amzn['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords.words('english')]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df_amzn.to_csv('amzn_title_text_label.csv', index=False)

In [ ]:
nltk.download('punkt')
from nltk.tokenize import word_tokenize

df_amzn['text_without_stopwords'] = df_amzn['text_without_stopwords'].apply(lambda x: ' '.join([word for word in word_tokenize(x) if word not in string.punctuation]))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
df_amzn

,title,text,direction,text_without_stopwords
0,6 things to know before the opening bell,click chart for more in-depth data. 1. big tec...,-1,click chart in-depth data 1. big tech senate b...
1,"Netflix And The Competition - Netflix, Inc. (N...",investment thesis netflix ( nflx ) is a remark...,-1,investment thesis netflix nflx remarkable serv...
2,Pandora Stock Gets a Boost — Buy Now,mannkind surges more than 50% >>> read more\nb...,-1,mannkind surges 50 read breaking news sponsore...
3,MARKET SNAPSHOT: Tech Tumble Weighs On Stocks;...,"by mark decambre, marketwatch , ryan vlastelic...",-1,mark decambre marketwatch ryan vlastelica tech...
4,"New Month, Same Old Story: Trade Fears Persist...","it may be a new month, but the same trade conc...",-1,may new month trade concerns haunted market ye...
...,...,...,...,...
19074,"Microsoft Roundup: Hardware Leak, Cashier-less...",leaked documents showed what could be microsof...,-1,leaked documents showed could microsoft ’ s ms...
19075,General Electric's Fall From Grace: Don't Fret...,june 19th was a dark day in the history books ...,-1,june 19th dark day history books general elect...
19076,"Beware of the Russell 2000, but should you buy...","by thomas h. kee jr.\nthe russell 2000 is hot,...",-1,thomas h. kee jr. russell 2000 hot lower liqui...
19077,"Beware of the Russell 2000, but should you buy...",there has been a bifurcation in the u.s. stock...,-1,bifurcation u.s. stock market recently unusual...


In [ ]:
common_words=pd.Series(''.join(df_amzn['text_without_stopwords']).split()).value_counts()
common_words

’           127903
s           125473
's           91005
apple        88884
stock        74062
             ...  
2,678.9          1
15:20:32         1
35,609           1
electrek         1
1746et           1
Length: 343245, dtype: int64

In [ ]:
common_words = common_words.drop(labels=[x for x in common_words.index if common_words[x] < 100])

In [ ]:
common_words

’               127903
s               125473
's               91005
apple            88884
stock            74062
                 ...  
ch0312806448       100
boat               100
yieldco            100
-0.16              100
clear=             100
Length: 10344, dtype: int64

In [ ]:
common_words = common_words.drop(labels=['’','s',"'s"])

In [ ]:
common_words

apple           88884
stock           74062
market          71762
company         66817
shares          64486
                ...  
ch0312806448      100
boat              100
yieldco           100
-0.16             100
clear=            100
Length: 10341, dtype: int64

In [ ]:

texts=[]
labels=[]
def removeWords(words):
    final_words=[]
    for word in words:
        if word in common_words:
            final_words.append(word)
    return ' '.join(final_words)

df_amzn['processedText'] = df_amzn['text_without_stopwords'].apply(lambda x: removeWords(word_tokenize(x)))



In [ ]:
df_amzn

,title,text,direction,text_without_stopwords,processedText
0,6 things to know before the opening bell,click chart for more in-depth data. 1. big tec...,-1,click chart in-depth data 1. big tech senate b...,click chart in-depth data 1. big tech senate b...
1,"Netflix And The Competition - Netflix, Inc. (N...",investment thesis netflix ( nflx ) is a remark...,-1,investment thesis netflix nflx remarkable serv...,investment thesis netflix nflx remarkable serv...
2,Pandora Stock Gets a Boost — Buy Now,mannkind surges more than 50% >>> read more\nb...,-1,mannkind surges 50 read breaking news sponsore...,surges 50 read breaking news sponsored home st...
3,MARKET SNAPSHOT: Tech Tumble Weighs On Stocks;...,"by mark decambre, marketwatch , ryan vlastelic...",-1,mark decambre marketwatch ryan vlastelica tech...,mark decambre marketwatch ryan vlastelica tech...
4,"New Month, Same Old Story: Trade Fears Persist...","it may be a new month, but the same trade conc...",-1,may new month trade concerns haunted market ye...,may new month trade concerns market year remai...
...,...,...,...,...,...
19074,"Microsoft Roundup: Hardware Leak, Cashier-less...",leaked documents showed what could be microsof...,-1,leaked documents showed could microsoft ’ s ms...,leaked documents showed could microsoft msft f...
19075,General Electric's Fall From Grace: Don't Fret...,june 19th was a dark day in the history books ...,-1,june 19th dark day history books general elect...,june dark day history books general electric n...
19076,"Beware of the Russell 2000, but should you buy...","by thomas h. kee jr.\nthe russell 2000 is hot,...",-1,thomas h. kee jr. russell 2000 hot lower liqui...,thomas jr. russell 2000 hot lower liquidity me...
19077,"Beware of the Russell 2000, but should you buy...",there has been a bifurcation in the u.s. stock...,-1,bifurcation u.s. stock market recently unusual...,u.s. stock market recently unusual because typ...


In [ ]:
def tokenize(x):
    x_tk = Tokenizer()
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk


In [ ]:
#title_amzn=df_amzn['title'].tolist()
text_amzn=df_amzn['processedText'].tolist()
label_amzn=df_amzn['direction'].tolist()


#print(title_amzn[0])
print(text_amzn[0])
print(label_amzn[0])

click chart in-depth data 1. big tech senate big tech executives coming capitol hill wednesday testify election meddling facebook fb coo sheryl sandberg jack dorsey ceo twitter twtr appear senate intelligence committee discuss actions companies taken foreign influence campaigns targeting 2018 midterm elections committee requested high-profile executive google owns youtube also appear testify google googl parent company alphabet however declined send one top people 2. emerging markets fears emerging markets currencies tumbled dollar wednesday mexican south indian among biggest losers rising us interest rates political global trade war recent months exposed multiple emerging markets turkey argentina hardest hit investors worried adding fears markit index measures emerging markets manufacturing fell fresh low august watch cnnmoney now today p.m. et 3. global stocks overview us stock futures pointing lower european markets opened down exchanges london frankfurt paris dropping roughly 0.5 a

In [ ]:
#title_tokenized_amzn, title_tokenizer_amzn=tokenize(title_amzn)
text_tokenized_amzn, text_tokenizer_amzn=tokenize(text_amzn)
#title_padded_amzn=pad_sequences(title_tokenized_amzn, padding='post')
text_padded_amzn=pad_sequences(text_tokenized_amzn, padding='post')

In [ ]:
class_label_amzn = pd.get_dummies(label_amzn).values

In [ ]:
class_label_amzn[0:5]

array([[1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0]], dtype=uint8)

In [ ]:
#print(title_tokenized_amzn[0])
print(text_padded_amzn[0])

[480 247 481 ...   0   0   0]


In [ ]:
from sklearn.model_selection import train_test_split
# trainY = pd.get_dummies(label.values).values
# print(trainY.shape)
#X_train, X_test, Y_train, Y_test = train_test_split(title_padded_amzn, class_label_amzn, test_size=0.2, random_state=56)
X_train, X_test, Y_train, Y_test = train_test_split(text_padded_amzn, class_label_amzn, test_size=0.2, random_state=56)

In [ ]:
TrainX = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
TestX = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
TrainX = TrainX.astype('float64')
TestX = TestX.astype('float64')

In [ ]:
TrainX[0:5]

array([[[9.300e+01, 2.600e+02, 3.839e+03, ..., 0.000e+00, 0.000e+00,
         0.000e+00]],

       [[4.354e+03, 1.413e+03, 1.194e+03, ..., 0.000e+00, 0.000e+00,
         0.000e+00]],

       [[1.770e+02, 4.000e+00, 3.300e+01, ..., 0.000e+00, 0.000e+00,
         0.000e+00]],

       [[3.400e+01, 5.800e+01, 2.446e+03, ..., 0.000e+00, 0.000e+00,
         0.000e+00]],

       [[3.000e+01, 3.200e+01, 5.000e+00, ..., 0.000e+00, 0.000e+00,
         0.000e+00]]])

In [ ]:
TrainX.shape

(15263, 1, 12487)

In [ ]:
from tensorflow.keras.layers import Bidirectional, LSTM, Dense

In [ ]:

layers = [LSTM(256,input_shape=(1,TrainX.shape[2]),dropout=0.2), Dense(2,activation='sigmoid')]
model = tf.keras.models.Sequential(layers)
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.SGD(),metrics=['accuracy'])
model.fit(TrainX, Y_train, epochs=40, batch_size=64, verbose=2,validation_split=0.2)

Epoch 1/40
191/191 - 23s - loss: 0.7131 - accuracy: 0.5052 - val_loss: 0.7007 - val_accuracy: 0.4946 - 23s/epoch - 118ms/step
Epoch 2/40
191/191 - 20s - loss: 0.6988 - accuracy: 0.5099 - val_loss: 0.6975 - val_accuracy: 0.5100 - 20s/epoch - 105ms/step
Epoch 3/40
191/191 - 20s - loss: 0.6966 - accuracy: 0.5170 - val_loss: 0.6988 - val_accuracy: 0.4946 - 20s/epoch - 105ms/step
Epoch 4/40
191/191 - 20s - loss: 0.6954 - accuracy: 0.5138 - val_loss: 0.6959 - val_accuracy: 0.4966 - 20s/epoch - 105ms/step
Epoch 5/40
191/191 - 20s - loss: 0.6936 - accuracy: 0.5206 - val_loss: 0.6991 - val_accuracy: 0.4946 - 20s/epoch - 106ms/step
Epoch 6/40
191/191 - 20s - loss: 0.6943 - accuracy: 0.5159 - val_loss: 0.7016 - val_accuracy: 0.4867 - 20s/epoch - 104ms/step
Epoch 7/40
191/191 - 20s - loss: 0.6935 - accuracy: 0.5150 - val_loss: 0.6959 - val_accuracy: 0.5021 - 20s/epoch - 104ms/step
Epoch 8/40
191/191 - 20s - loss: 0.6932 - accuracy: 0.5189 - val_loss: 0.6950 - val_accuracy: 0.4985 - 20s/epoch - 104

In [ ]:
model.evaluate(TestX, Y_test, verbose=2)

120/120 - 3s - loss: 0.6947 - accuracy: 0.5183 - 3s/epoch - 27ms/step


[0.6946567893028259, 0.5183438062667847]